In [1]:
import pandas as pd
import numpy as np
import jieba.analyse
import re
import json

In [2]:
## 原来的有点Bug，修改了
stop = open('hit_stopwords.txt', 'r+', encoding='utf-8')
stop_words = stop.read().split("\n") + [' ']
stop.close()
def skip_stop_words(text):
    res =[]
    for i in text:
        for word in i:
            if word not in stop_words:
                res.append(word)
            
    return ' '.join(res)

In [3]:
df = pd.read_csv('sample1.txt', header=0, sep='\t')

In [4]:
df.sort_values(by = 'customer_segment_org', inplace = True)

In [5]:
df['text_list'] = df['text_list'].map(lambda x:json.loads(x))
df['text_list'] = df['text_list'].map(lambda x:skip_stop_words(x))

In [6]:
df

,customer_segment_org,text_list
5,A-富裕,已经 申请 延期 申请 延期 申请 延期 多谢 转 人工 你好 好 不用 没有 理财 全部 ...
4,B1-优质中产,逾期 几天 知道 之前 处 处理 一笔 上上个月 一期 应该 几天 搞进去 好 好 现在 现...
10,B2-成长中产,还 没 借到 钱 现在 应该 一千块 先 还 掉 再 看 一下 不 知道 不能 理解 差 将...
0,B3-成长中产,你好 说话 京东 都 给我发 短信 说 已经 说 一下 已经 给我发 短信 不让 说 说 说...
6,C1-优质白领,听 得到 现在 没 出去 没有 钱 付款 三月份 现在 做点 生意 之前 亏了 鞋子 应该 ...
8,C2-成长白领,号码 没有 收到 验证码 登 一下 六五 二 一八九 号码 我用 号码 登登 不 上去 九五...
2,C3-成长白领,不是 不是 那天 说 不是 上个月 我用 身份证 办 上个月 三十 号 刚刚 办 还 办理 ...
9,D1-优质蓝领,你好 说 好 上次 不是 协商 到时候 说 没 说 不是 现在 不是 说 暂时 处理 不了 ...
1,D2-成长蓝领,说 知道 找 没有 上班 好 工作 现在 工作 不是 年底 发 工作 上 好 再见 想想 办...
7,D3-成长蓝领,好 你好 你好 事 不 不是 意思 账户 京东 金条 意思 这是 意思 不 知道 意思 说 ...


In [7]:
#----
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
corpus = list(df['text_list'])

In [8]:
#---- 自定义IDF词典

In [9]:
import math
idf_dic = {}
#data_content是带分析文本，一个demo：如下图
doc_count = len(corpus) # 总共有多少篇文章

In [10]:
for i in range(len(corpus)):
    new_content = corpus[i].split(' ')
    for word in set(new_content):
        idf_dic[word] = idf_dic.get(word, 0.0) + 1.0
        # 此时idf_dic的v值：有多少篇文档有这个词，就是多少
for k,v in idf_dic.items():
    w = k
    p = '%.10f' % (math.log(doc_count / (1.0 + v))) # 结合上面的tf-idf算法公式
    if w > u'\u4e00' and w <= u'\u9fa5': # 判断key值全是中文
        idf_dic[w] = p
        
with open('tfidf_dict.txt','w',encoding='utf-8') as f:
    for k in idf_dic:
        if k != '\n':
            f.write(k + ' ' + idf_dic[k] + '\n') #写入txt文件，注意utf-8，否则jieba不认

In [11]:
#----

In [12]:
jieba.analyse.set_idf_path('tfidf_dict.txt')

In [13]:
tfidf = {}
count = []
for i in range(len(df)):
    words = list(jieba.analyse.extract_tags(df.iloc[i, 1], topK = 500, withWeight = False))
    tfidf[df.iloc[i, 0]] = words
    for word in words:
        count.append({'seg':df.iloc[i, 0], 'word':word, 'count':df.iloc[i, 1].count(word)})

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.967 seconds.
Prefix dict has been built successfully.


In [14]:
df_tfidf = pd.DataFrame(tfidf)
df_count = pd.DataFrame(count)
df_count = df_count.groupby(['seg', 'word']).sum().reset_index()

In [15]:
df_tfidf.to_csv('custgroup_keyword_Top200_tfidf.txt', sep = '\t')
df_count.to_csv('custgroup_keyword_Top200_count.txt', sep = '\t')

In [27]:
df.iloc[9, 1].count('投资')

75

In [38]:
#----

In [1]:
segments = []
for index, row in df.iterrows():
    content = row[1]
    #TextRank 关键词抽取，只获取固定词性
    words = jieba.analyse.extract_tags(content, topK=50,withWeight=False)
    splitedStr = ''
    for word in words:
        # 记录全局分词
        segments.append({'word':word, 'count':1})
        splitedStr += word + ' '
dfSg = pd.DataFrame(segments)

In [ ]:
#----

In [40]:
tfidf = TfidfVectorizer()
response = tfidf.fit_transform(corpus)
feature_names = tfidf.get_feature_names()
response_matrix = response.toarray()
tfidf_k = np.argpartition(response_matrix, np.argmin(response_matrix, axis=0))[:, -1000:] ## 每个文档前1000关键词的index

ans = []
for i in range(len(df)):
    res = []
    for index in list(tfidf_k[i]):
        if feature_names[index] in corpus[i].split(' '):
            res.append(feature_names[index])
    ans.append(res)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
k = 3
model = KMeans(n_clusters=k)

model.fit(response_matrix)

In [ ]:
tsne = TSNE()
tsne.fit_transform(response_matrix) #进行数据降维

In [ ]:
tsne = pd.DataFrame(tsne.embedding_) #转换数据格式
tsne['label'] = model.labels_

In [11]:
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号
#不同类别用不同颜色和样式绘图
d = tsne[tsne['label'] == 0]
plt.plot(d[0], d[1], 'r.')
d = tsne[tsne['label'] == 1]
plt.plot(d[0], d[1], 'go')
d = tsne[tsne['label'] == 2]
plt.plot(d[0], d[1], 'b*')
plt.show()

NameError: name 'tsne' is not defined